# Set-up

In [5]:
cd /content/drive/MyDrive/COMP0087/allenNLP/Model_2

/content/drive/MyDrive/COMP0087/allenNLP/Model_2


In [ ]:
# %%shell
# pip install torch==1.7.1

In [6]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np
import spacy
import pandas as pd

In [3]:
%%shell
pip install allennlp==2.1.0 allennlp-models==2.1.0
# pip install allennlp

     |████████████████████████████████| 593kB 8.3MB/s 
     |████████████████████████████████| 409kB 17.2MB/s 
     |████████████████████████████████| 133kB 26.3MB/s 
     |████████████████████████████████| 12.8MB 233kB/s 
     |████████████████████████████████| 266kB 60.9MB/s 
     |████████████████████████████████| 122kB 48.9MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 1.2MB 47.9MB/s 
     |████████████████████████████████| 1.9MB 49.7MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 7.4MB 46.3MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 3.3MB 25.4MB/s 
     |████████████████████████████████| 870kB 50.8MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp37-cp37m-linux_x86_64.whl size=3388784 sha256=976b86e8e6c9ba5a10371ec3e11a24475ee2a7b033ec1cdec2016d1a21de3203

In [1]:
# imports from allennlp
from allennlp.models.archival import load_archive
from allennlp.common.util import JsonDict
from allennlp.data import Instance
from allennlp.predictors.predictor import Predictor
from allennlp.data.fields import LabelField
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer

from typing import List, Dict

from overrides import overrides

from allennlp.interpret.attackers import Attacker, InputReduction

from allennlp.interpret.saliency_interpreters import SimpleGradient

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from scipy.stats import kendalltau
from scipy.stats import spearmanr

In [7]:
import tagging

# Training

In [ ]:
# training model
!pwd; allennlp train --include-package tagging -s LSTM_output config_LSTM.jsonnet

/content/drive/My Drive/COMP0087/allenNLP/Model_2
2021-04-01 15:14:59,140 - ERROR - allennlp.common.plugins - Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'
2021-04-01 15:14:59,459 - INFO - allennlp.common.params - include_in_archive = None
2021-04-01 15:14:59,462 - INFO - allennlp.common.params - random_seed = 13370
2021-04-01 15:14:59,463 - INFO - allennlp.common.params - numpy_seed = 1337
2021-04-01 15:14:59,463 - INFO - allennlp.common.params - pytorch_seed = 133
2021-04-01 15:14:59,484 - INFO - allennlp.common.checks - Pytorch version: 1.7.1
2021-04-01 15:14:59,484 - INFO - allennlp.common.params - type = default
2021-04-01 15:14:59,485 - INFO - allennlp.common.params - dataset_reader.type = ag_reader
2021-04-01 15:14:59,486 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2021-04-01 15:14:59,486 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2021-04-01 15:14:59,486 - INFO - allen

# Predictions

In [8]:
@Predictor.register('ag_text_classifier')
class AGNewsClassifier(Predictor):
    """
    Predictor for any model that takes in a sentence and returns
    a single class for it.  In particular, it can be used with
    the [`BasicClassifier`](../models/basic_classifier.md) model.

    """

    def predict(self, sentence: str) -> JsonDict:
        return self.predict_json({"Description": sentence})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        """
        Expects JSON that looks like `{"sentence": "..."}`.
        Runs the underlying model, and adds the `"label"` to the output.
        """
        sentence = json_dict["Description"]
        reader_has_tokenizer = (
            getattr(self._dataset_reader, "tokenizer", None) is not None
            or getattr(self._dataset_reader, "_tokenizer", None) is not None
        )
        if not reader_has_tokenizer:
            tokenizer = SpacyTokenizer()
            sentence = tokenizer.tokenize(sentence)
        return self._dataset_reader.text_to_instance(sentence)

    @overrides
    def predictions_to_labeled_instances(
        self, instance: Instance, outputs: Dict[str, np.ndarray]
    ) -> List[Instance]:
        new_instance = instance.duplicate()
        label = np.argmax(outputs["probs"])
        new_instance.add_field("label", LabelField(int(label), skip_indexing=True))
        return [new_instance]

In [9]:
archive = load_archive("./LSTM_output/model.tar.gz")
model = archive.model
vocab = model.vocab

In [10]:
predictor = Predictor.from_archive(archive, 'ag_text_classifier')

In [20]:
input = "The Cleveland Indians pulled within one game of the AL Central lead by beating the Minnesota Twins, 7-1, Saturday night with home runs by Travis Hafner and Victor Martinez."

In [21]:
predictor.predict(sentence=input)

{'label': '2',
 'logits': [-2.927020788192749,
  5.316094875335693,
  -1.3947194814682007,
  -1.6728702783584595],
 'probs': [0.00026243284810334444,
  0.9976029992103577,
  0.0012147531379014254,
  0.0009197902400046587],
 'token_ids': [16,
  1447,
  3317,
  1351,
  731,
  64,
  102,
  8,
  2,
  2171,
  1388,
  188,
  27,
  1147,
  2,
  1050,
  2700,
  3,
  342,
  5,
  136,
  3,
  103,
  107,
  19,
  170,
  988,
  27,
  8015,
  27744,
  10,
  6822,
  2252,
  4],
 'tokens': ['The',
  'Cleveland',
  'Indians',
  'pulled',
  'within',
  'one',
  'game',
  'of',
  'the',
  'AL',
  'Central',
  'lead',
  'by',
  'beating',
  'the',
  'Minnesota',
  'Twins',
  ',',
  '7',
  '-',
  '1',
  ',',
  'Saturday',
  'night',
  'with',
  'home',
  'runs',
  'by',
  'Travis',
  'Hafner',
  'and',
  'Victor',
  'Martinez',
  '.']}

In [23]:
output = predictor.predict(input)
print(
    [
        (vocab.get_token_from_index(label_id, "labels"), prob)
        for label_id, prob in enumerate(output["probs"])
    ]
)

[('4', 0.00026243284810334444), ('2', 0.9976029992103577), ('1', 0.0012147531379014254), ('3', 0.0009197902400046587)]


# Interpret

In [11]:
@Predictor.register('ag_text_classifier_with_input_red')
class InputReductionTextClassifierPredictor(AGNewsClassifier):
    
    def predict_json(self, json_dict: JsonDict) -> JsonDict:
        predictor = AGNewsClassifier(self._model, self._dataset_reader)
        prediction = predictor.predict(sentence=json_dict['Description'])

        attacker = InputReduction(predictor)
        attack = attacker.attack_from_json(inputs=json_dict,
                                           input_field_to_attack='tokens',
                                           grad_input_field='grad_input_1',
                                           ignore_tokens=None)

        return {'prediction': prediction, 'input_reduction_output': attack}

In [12]:
predictor_with_input_red = Predictor.from_archive(archive, 'ag_text_classifier_with_input_red')

In [14]:
result = predictor_with_input_red.predict(input)
tokens = result['prediction']['tokens']

grad = SimpleGradient(predictor).saliency_interpret_from_json({'Description':input})
gradient_list = grad['instance_1']['grad_input_1']

# Visualisation

Source: https://adataanalyst.com/machine-learning/highlight-text-using-weights/

In [15]:
import html
import random
from IPython.core.display import display, HTML

In [16]:
# Prevent special characters like & and < to cause the browser to display something other than what you intended.
def html_escape(text):
    return html.escape(text)

In [17]:
max_alpha = 0.4 
highlighted_text = []
for i in range(len(tokens)):
    weight = gradient_list[i]
    highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
highlighted_text = ' '.join(highlighted_text)

In [18]:
display(HTML(highlighted_text))

In [19]:
print(display(HTML(highlighted_text)))

None


# Checklist

In [ ]:
%%shell
pip install checklist

     |████████████████████████████████| 12.1MB 252kB/s 
     |████████████████████████████████| 22.3MB 1.6MB/s 
     |████████████████████████████████| 174kB 51.8MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 5.6MB 52.3MB/s 
     |████████████████████████████████| 5.5MB 57.1MB/s 
     |████████████████████████████████| 419kB 51.1MB/s 
     |████████████████████████████████| 3.2MB 42.4MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
  Created wheel for checklist: filename=checklist-0.0.10-cp37-none-any.whl size=12164434 sha256=778ea9166bff4db74b0add99ba9c6083f10416d68dd8b8c16964d744d6eee04a
  Stored in directory: /root/.cache/pip/wheels/c6/a2/02/f075a28620a8370dd79df36bb486095d465e7870623cea6b4e
  Created wheel for patternfork-nosql: filename=patternfork_nosql-3.6-cp37-none-any.whl size=22332807 sha256=c6e54fd57ca87f2902dd87abe35b581a138ce1b9e6710e38db675dfae2f0a56f
  Stored in directory: /root/.cache/pip/wheels/dc/0d/ae

In [ ]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

In [ ]:
test_set = pd.read_json('/content/drive/MyDrive/COMP0087/data/test.jsonl', orient='records', lines=True)

In [ ]:
sample = []
for row in test_set.head(10).itertuples():
  sample.append(row[3])

In [ ]:
for sentence in sample:
  result = predictor_with_input_red.predict(sentence)
  tokens = result['prediction']['tokens']

  grad = SimpleGradient(predictor).saliency_interpret_from_json({'Description':sentence})
  gradient_list = grad['instance_1']['grad_input_1']
  max_alpha = 0.4 
  highlighted_text = []
  for i in range(len(tokens)):
      weight = gradient_list[i]
      highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
  highlighted_text = ' '.join(highlighted_text)
  print(display(HTML(highlighted_text)))

None


None


None


None


None


None


None


None


None


None


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
pdata = list(nlp.pipe(sample))

In [ ]:
Perturb.strip_punctuation(pdata[0])

"Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul"

In [ ]:
Perturb.contractions(sample[0])[0]

"Unions representing workers at Turner   Newall say they're 'disappointed' after talks with stricken parent firm Federal Mogul."

In [ ]:
for i in range(10):
  sentence = sample[i]
  if Perturb.contractions(sample[i]) != []:
  
    result = predictor_with_input_red.predict(sentence)
    tokens = result['prediction']['tokens']

    grad_orig = SimpleGradient(predictor).saliency_interpret_from_json({'Description':sentence})
    gradient_list_orig = np.array(grad_orig['instance_1']['grad_input_1'])

    max_alpha = 0.4 
    highlighted_text = []
    for i in range(len(tokens)):
        weight = gradient_list_orig[i]
        highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
    highlighted_text = ' '.join(highlighted_text)
    print(display(HTML(highlighted_text)))

    perturbed_sentence = Perturb.contractions(sentence)[0]

    result = predictor_with_input_red.predict(perturbed_sentence)
    tokens = result['prediction']['tokens']

    grad_pert = SimpleGradient(predictor).saliency_interpret_from_json({'Description': perturbed_sentence})
    gradient_list_pert = np.array(grad_pert['instance_1']['grad_input_1'])

    max_alpha = 0.4 
    highlighted_text = []
    for i in range(len(tokens)):
        weight = gradient_list_pert[i]
        highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
    highlighted_text = ' '.join(highlighted_text)
    print(display(HTML(highlighted_text)))

None


None


None


None


KeyboardInterrupt: ignored

In [1]:
gradient_list_pert

NameError: ignored

In [ ]:
rank_orig = np.argsort(-gradient_list_orig)
ranked_elements_orig = np.empty(len(rank_orig))
rank = 1
for i in range(len(rank_orig)):
  ranked_elements_orig[rank_orig[i]] = rank
  rank += 1
ranked_elements_orig

array([24., 47., 19., 31., 40.,  2.,  6.,  3., 22., 49., 25., 48., 46.,
       50., 36., 45., 18.,  1., 10., 54., 39.,  7., 14., 27., 34., 13.,
       37., 12., 35., 41., 29., 30., 32., 56.,  8., 28., 16., 42., 23.,
       33.,  5., 43., 11., 38., 51., 52., 44.,  4., 21., 26., 15., 20.,
       17.,  9., 55., 53.])

In [ ]:
rank_pert = np.argsort(-gradient_list_pert)
ranked_elements_pert = np.empty(len(rank_pert))
rank = 1
for i in range(len(rank_pert)):
  ranked_elements_pert[rank_pert[i]] = rank
  rank += 1
ranked_elements_pert

array([32., 41.,  6., 13., 24.,  2.,  4.,  3.,  5., 40., 27., 31., 48.,
       51., 35., 52., 26.,  1., 15., 46., 49., 16.,  9., 33., 39., 14.,
       36., 11., 38., 47., 30., 29., 34., 54.,  7., 25., 19., 42., 20.,
       37., 10., 44., 18., 45., 50., 56., 43.,  8., 23., 28., 17., 22.,
       21., 12., 53., 55.])

In [ ]:
kendalltau(rank_pert, rank_orig)

KendalltauResult(correlation=0.18571428571428575, pvalue=0.04324716848226369)

In [ ]:
spearmanr(rank_pert, rank_orig)

SpearmanrResult(correlation=0.27771701982228303, pvalue=0.03823479957213016)